#### m6A motif

In [3]:
### extend to window
# m6a_dir = "/Charles/project/ASm6A/ASm6A/"
# os.chdir(m6a_dir)
# #
# result_dir = "motif/ASm6A/window/"
# os.system("mkdir -p %s" % result_dir)
# #
# bed_list = glob.glob("*.bed")
# for bed in bed_list:
#     # os.system("intersectBed -wo -a %s -b %s | awk -v OFS="\t" '{print $1,$2,$3,"*","*",$10}' | sort -k1,2 | uniq > ${prefix}_bestpeaks.bed")
#     df = pd.read_table(bed, sep="\t", header=None)
#     df.iloc[:,1] = df.iloc[:,1] - 4
#     df.iloc[:,2] = df.iloc[:,2] + 4
#     res = os.path.join(result_dir, os.path.basename(bed))
#     df.to_csv(res, sep="\t", header=False, index=False)
    
### get strand info
# win_dir = "/Charles/project/ASm6A/ASm6A/motif/ASm6A/window/"
win_dir = "/home/galaxy/data/RBP/POSTAR/POSTAR_bed/motif/window/"
os.chdir(win_dir)
result_dir = "../add_strand/"
os.system("mkdir -p %s" % result_dir)
gene = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/gencode.v19.annotation_Gene.bed"
#
bed_list = glob.glob("*.bed")
for bed in bed_list:
    res = os.path.join(result_dir, os.path.basename(bed))
    os.system("intersectBed -a %s -b %s -wa -wb | sort -u > %s" % (bed, gene, res))

### get fasta
# stra_dir = "/Charles/project/ASm6A/ASm6A/motif/ASm6A/add_strand/"
os.chdir("../add_strand/")
##
! for x in *.bed;do less $x | awk '{OFS="\t"}{print $1,$2,$3,".",".",$9}' | sort -k1,1 -k2,2n | uniq > ${x}2;done
##
result_dir = "../fasta/"
os.system("mkdir -p %s" % result_dir)
fasta = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa"
#
bed_list = glob.glob("*.bed2")
for bed in bed_list:
    res = os.path.join(result_dir, os.path.basename(bed).replace(".bed2",'.fa'))
    os.system("bedtools getfasta -s -fi %s -bed %s -fo %s" % (fasta, bed, res))
    
# #### get alt fasta
# base_dict = {"A": "T", "T":"A", "C":"G", "G":"C"}
# fa_dir = "/Charles/project/ASm6A/ASm6A/motif/ASm6A/fasta/"
# os.chdir(fa_dir)
# result_dir = "../alt_fasta/"
# os.system("mkdir -p %s" % result_dir)
# #
# raw_dir = "/Charles/project/ASm6A/ASm6A/motif/ASm6A/add_strand/"
# bed_list = glob.glob("%s/*.bed" % raw_dir)
# for bed in bed_list:
#     # print(bed)
#     prefix, title_dict = os.path.basename(bed).split(".")[0], {}
#     df = pd.read_table(bed, header=None)
#     # >chr10:102116306-102116315(+)    ### 注意修改这一行链的位置信息
#     df['title'] = ">" + df.iloc[:,0] + ":" + df.iloc[:,1].astype(str) + "-" + df.iloc[:,2].astype(str) + "(" + df.iloc[:,11] + ")"
#     df["alleles"] = df.iloc[:,3].str.split(";").str[0]
#     title_dict = dict(zip(df['title'], df['alleles']))
#     # print(title_dict['>chr10:101379275-101379284(-)']) 
#     #
#     fasta = os.path.join(fa_dir, prefix+".fa")
#     fa_dict = {}
#     with open(fasta, 'r') as f:
#         contents = f.readlines()
#         for i in range(len(contents)-1):
#             fa_dict[contents[i].strip()] = contents[i+1].strip()
#             i += 1
#     # print(fa_dict['>chr10:101379275-101379284(-)']) # GTGATTGGA
#     #
#     result_file = os.path.join(result_dir, os.path.basename(fasta))
#     res_list = []
#     for title, sequ in fa_dict.items():
#         if title in title_dict:
#             alleles = title_dict[title]
#             if "(+)" in title:
#                 sequ = list(sequ.upper())
#                 assert sequ[4] == alleles.split(">")[0]
#                 sequ[4] = alleles.split(">")[1]
#                 res_list.append(title)
#                 res_list.append("".join(sequ))
#             elif "(-)" in title:
#                 sequ = list(sequ.upper())
#                 assert sequ[4] == base_dict[alleles.split(">")[0]]
#                 sequ[4] = base_dict[alleles.split(">")[1]]
#                 res_list.append(title)
#                 res_list.append("".join(sequ))
#     with open(result_file, 'w') as fw:
#         for line in res_list:
#             fw.write(line + "\n")

#### identifying m6A motif

In [3]:
# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/peak/motif/")
# os.chdir("/Charles/project/ASm6A/ASm6A/motif/")
os.chdir("/Charles/project/ASm6A/ASm6A/motif/ASm6A/add_strand/total_homer_nt6/")

# bed_file="/home/galaxy/project/alleleSpecific_analysis/data/shFUS/peak/macs2_Nc1_normalized.bed"
# bed_file = "m6A.bed"
bed_file = "/Charles/project/ASm6A/ASm6A/motif/ASm6A/add_strand/total_homer_nt6/ASm6A.bed"
gtf="/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19_only22.gtf"
fasta="/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa"
# prefix="Nc1"
prefix = "ASm6A"
!sort -k5,5 -g ${bed_file}| head -10000 | awk '{ print $1"\t"$2"\t"$3}' > ${prefix}.location
!intersectBed -wo -a ${prefix}.location -b $gtf | awk -v OFS="\t" '{print $1,$2,$3,"*","*",$10}' | sort -k1,2 | uniq > ${prefix}_bestpeaks.bed
!bedtools getfasta -s -fi $fasta -bed ${prefix}_bestpeaks.bed -fo ${prefix}_bestpeaks.fa
# ame -oc ${prefix}_ame ${prefix}_bestpeaks.fa $RRACH_motif
!findMotifsGenome.pl ${prefix}_bestpeaks.bed $fasta ${prefix}_homer -len 6 -rna -p 15 # ${THREAD_NUM:=1}
#/home/software/meme-5.0.2/scripts/dreme -k 7 -oc ${prefix}_dreme -p ${prefix}_bestpeaks.fa -rna

^C

Tool:    bedtools intersect (aka intersectBed)
Version: v2.27.1
Summary: Report overlaps between two feature files.

Usage:   bedtools intersect [OPTIONS] -a <bed/gff/vcf/bam> -b <bed/gff/vcf/bam>

	Note: -b may be followed with multiple databases and/or 
	wildcard (*) character(s). 
Options: 
	-wa	Write the original entry in A for each overlap.

	-wb	Write the original entry in B for each overlap.
		- Useful for knowing _what_ A overlaps. Restricted by -f and -r.

	-loj	Perform a "left outer join". That is, for each feature in A
		report each overlap with B.  If no overlaps are found, 
		report a NULL feature for B.

	-wo	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlaps restricted by -f and -r.
		  Only A features with overlap are reported.

	-wao	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlapping features restricted by -f and -r.
		  However, A 

#### 1. RRACH sequence within peak¶
#### 2. motif enrichment in ASm6A

In [6]:
import math

os.chdir("/Charles/project/ASm6A/ASm6A/motif/m6A/RRACH/")
gene = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/gencode.v19.annotation_Gene.bed"
m6a = "../m6A.bed"
res = "Allsamples_hg19.overlapgene"
os.system("intersectBed -a %s -b %s -wa -wb | sort -u > %s" % (m6a, gene, res))
! less Allsamples_hg19.overlapgene | awk '{OFS="\t"}{print $1,$2,$3,".",".",$9}' | sort -k1,2 | uniq > Allsamples_hg19.overlapgene2
fasta = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa"
bed = res + "2"
res = "Allsamples_hg19.fa"
os.system("bedtools getfasta -s -fi %s -bed %s -fo %s" % (fasta, bed, res))

motif = "/home/galaxy/project/alleleSpecific_analysis/data/motif/m6A_motif.homer"
fa = res
res = "Allsamples_hg19.txt"
os.system("findMotifs.pl %s fasta homer/ -find %s > %s" % (fa, motif, res))

txt = "Allsamples_hg19.txt"
result_file = "Allsamples_hg19.sites"
df = pd.read_table(txt, sep="\t")
df['chr'] = df['FASTA ID'].str.split(":").str[0]
df['strand'] = df['FASTA ID'].str.split("(").str[1].str.split(")").str[0]
df['start'] = df['FASTA ID'].str.split(":").str[1].str.split("-").str[0].astype(int)
df['end'] = df['FASTA ID'].str.split(":").str[1].str.split("-").str[1].str.split("(").str[0].astype(int)
###
df_same = df[df['Sequence'] == df['Motif Name']]
df_p = df_same[df_same['strand'] == "+"]
# df_p['site'] = (df_p['end'] - 4) + df_p['Offset'] +2
df_p['site'] = (df_p['start']+1+df_p['end'])/2 + df_p['Offset'] +2
#
df_n = df_same[df_same['strand'] == "-"]
# df_n['site'] = (df_n['end'] - 4) - df_n['Offset'] -2
df_n['site'] = (df_n['start']+1+df_n['end'])/2 - df_n['Offset'] -2
df_same = pd.concat([df_p, df_n])
###
df_reverse = df[df['Sequence'] != df['Motif Name']]
df_p = df_reverse[df_reverse['strand'] == "+"]
# df_p['site'] = (df_p['end'] - 4) + df_p['Offset'] -2
df_p['site'] = (df_p['start']+1+df_p['end'])/2 + df_p['Offset'] -2
#
df_n = df_reverse[df_reverse['strand'] == "-"]
# df_n['site'] = (df_n['end'] - 4) - df_n['Offset'] +2
df_n['site'] = (df_n['start']+1+df_n['end'])/2 - df_n['Offset'] +2
df_reverse = pd.concat([df_p, df_n])

df = pd.concat([df_same, df_reverse])
# print(len(df_same), len(df_reverse), len(df))
site_list = df['site'].tolist()
site_list = [math.ceil(x) for x in site_list]
df['site'] = site_list
# df['s'] = df['site'] - 1
df['start'] = df['site'] - 3
df['end'] = df['site'] + 2
# df[['chr','s','site','FASTA ID']].to_csv(result_file, sep="\t", header=False, index=False)
df[['chr','start','end','FASTA ID']].to_csv(result_file, sep="\t", header=False, index=False)


### filter by letter: a,A,T,t
! bedtools getfasta -fi /home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa -bed Allsamples_hg19.sites > Allsamples_hg19.sites.fa
# letter_list = ["A", "a", "T", "t"]
motif_list = ['AAACC', 'GGTCT', 'AGTCC', 'AGTTC', 'TGTTC', 'TGTTT', 'AAACT', 'GGACC', 'AGTTT', 'AGACT','GAACA',
 'AGACC', 'GAACC', 'TGTCC', 'GGACT', 'GAACT', 'AGACA', 'AAACA', 'AGTCT', 'GGTTT', 'GGACA', 'GGTCC', 'TGTCT',
 'GGTTC']
result_list = []
with open("Allsamples_hg19.sites.fa", 'r') as f:
    contents = f.readlines()
    for i in range(len(contents)-1):
        if contents[i+1].strip().upper() in motif_list:
            result_list.append(contents[i])
            
with open("RRACH.bed", 'w') as fw:
    for x in result_list:
        chro = x.split(">")[1].split(":")[0]
        if (chro != "chrX") and (chro != "chrY"):
            start = x.split(":")[1].split("-")[0]
            end = x.split("-")[1]
            fw.write("%s\t%s\t%s" % (chro, start, end))

/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

388426 333885 722311
